# ChatGPT and OpenAI in Python
<!-- requirement: images/llm.png -->
<!-- requirement: images/apikey_1.png -->
<!-- requirement: chatgpt_template.env -->
<!-- requirement: data/ -->

ChatGPT is an advanced AI language model created by OpenAI. With a vast range of training data, it is designed to generate human-like text responses based on prompts and inquiries. Its capabilities span numerous topics, making it a very useful tool for generating natural language content.

It can perform a wide range of tasks, including:
- **Answering Questions**: provides information and answers on a wide variety of topics such as general knowledge, science, history, technology, etc.
- **Language Translation**: translates text from one language to another.
- **Text Generation**: generates human-like text for various purposes, such as writing essays, stories, poems, or even code snippets.
- **Spell Checking and Grammar Correction**: helps identify and correct spelling mistakes or grammatical errors in text.
- **Sentiment Analysis**: analyzes the sentiment expressed in a piece of text, determining whether it is positive, negative, or neutral.
- **Summarization**: generates concise summaries of long articles or documents, providing the key points and main ideas.
- **Content Creation**: helps generate ideas and content for creative writing, marketing copy, product descriptions, and more.
- **Conversational Agent**: engages in interactive conversations, answering questions, providing suggestions, and engaging in dialogues on various topics.


The text above is a testimony to the tasks ChatGPT can perform, as it was generated by ChatGPT when asked to describe itself and what it can be used for. If you haven't yet, feel free to play around with the [web version of ChatGPT](https://chat.openai.com/) and see for yourself what it can (and can not) do. As of the time of this writing the free web version of ChatGPT runs a GPT-3.5 model, but OpenAI has developed an upgraded GPT-4 version, which is at this moment available under paid access through ChatGPT Plus.


For the purpose of this demonstration, we will use ChatGPT on yet another task - extracting information from a bunch of legal documents.

## Example 1: extracting information from gift agreements

We have a bunch of legal files, more precisely gift agreements that contain information about who gifted whom what. For  each agreement we would like to extract the names of the donor and recipient as well as what was gifted. If you browse through the files, you will see that all of the information is there, but accomplishing this task by hand would be tedious. We will instead use ChatGPT to help us.

We will start by using the web version of ChatGPT, so let's do that first. We can give it a prompt like "Extract who gifted whom what in the following agreement." and paste the text of one of the gift agreements.

As you can probably see it works well and gives us the information we want from all the different gift agreements. 

How does it do that?

## Transformers: machine learning model behind the magic

ChatGPT, or GPT (Generative Pre-trained Transformer), is a large language model that uses a deep learning architecture called a Transformer. Transformers are a type of deep learning architecture that have significantly advanced natural language processing (NLP) tasks. They were introduced in the groundbreaking paper "Attention Is All You Need" by Vaswani et al. in 2017 and have since become widely used in various domains, including language translation, text generation, question answering, and more. 

In a nutshell a transformer model is trained to **predict the likelihood of the next word** given the previous sequence of words. GPT is trained on a massive amount of text data from the internet, including books, articles, websites, and more. This diverse dataset helps the model learn to generate grammatically correct sequences, state facts, and even display what looks like reasoning abilities.

Going into the details of the model is not the focus of this demonstration, so let's just highlight that the key ingredients of transformer models that resulted in them massively outperforming models that came before are:
- **Attention mechanism**: with very long term memory, the model has the capability to focus on different parts of the (potentially extremely long) input sentence as it generates the output.
- **Self-attention**: allows the model to gather relevant meaning of a word from the words around it and produce more accurate representations of each word. Example: the word "Python" will have a different representation in sentence "My python turned 2 years old today." compared to sentence "Python is my favorite language."

![LLM](images/llm.png)

The transformer architecture is composed of an **encoder** and **decoder** parts. The encoder part takes in the input sequence of words (or tokens) and transforms it into a rich representation that captures the meaning and context of the input. That information is then passed to the decoder that is responsible for generating the output sequence based on the information it receives from the encoder. While the decoder in a transformer model receives the representation from the encoder as its primary input, during inference, it also incorporates its own past model outputs as input for subsequent word generation.

As we mentioned, during the pre-training phase, GPT learns to predict the next token in a sequence of tokens based on the patterns it has learned from the training data. But after pre-training, the model can also undergo a process called fine-tuning. During this process it is further trained on more specific tasks with labeled examples. Fine-tuning allows the model to adapt to a particular use case, such as chat-based conversations.

## OpenAI API - using ChatGPT programmatically

Now that we have convinced ourselves that we can solve our task with ChatGPT and have seen a bit about how it works, we would like to use it programmatically. Using the web version is fine but has some limitations, for example if we want to integrate GPT's capabilities into other applications or we simply want to process a large amount of gift contracts, we don't want to do this by copy pasting to the web interface. We'd prefer to use the API.

### Setting things up

OpenAI provides a [`openai`](https://github.com/openai/openai-python) Python library that will enable us to do this easily. All we need to do is make sure we have the library installed and we will need an API key. If you do not have yours yet, you can go to [this website](https://platform.openai.com/account/api-keys) and generate your API key. In case you haven't generated one yet, you will see the following:
![generateAPIkey 1](images/apikey_1.png)

After clicking on "Create new secret key", you can give it a name and a new API key will be generated. Make sure to copy it and keep it safe on your computer. We recommend putting it into a .env file. We created a `chatgpt_template.env` file for you here, so you can see what the file should look like. Fill your API key in the template and rename the file to `chatgpt.env`. Now we can load the key as an environment variable as you see below. 

In [ ]:
import os
import openai
from dotenv import load_dotenv
_ = load_dotenv('chatgpt.env') 

openai.api_key  = os.getenv('OPENAI_API_KEY')

Now we are ready to make some requests. Note that if you do not add any billing options to your OpenAI account, you are using the free trial, which has a limited amount of free trial credits. You can check your usage [here](https://platform.openai.com/account/usage).

### Chat completions vs completions API

The API has two modes you can use. One is [`completions`](https://platform.openai.com/docs/guides/gpt/completions-api) which provides the response for a single prompt. The other is [`chat completions`](https://platform.openai.com/docs/guides/gpt/chat-completions-api) which  provides a response to a given dialog. This means that in chat mode the input is not one single prompt, but it instead requires input in a specific format that includes the conversation history. The chat API can be used with a single prompt as well however. The main difference between these APIs at this moment are underlying GPT models that are available in each (see [here](https://platform.openai.com/docs/guides/gpt/chat-completions-vs-completions)). For our example of extracting information from gift contracts, we do not need the chat options, but because the chat completions API is the interface to the most capable model and the most cost effective model, we will use that. 

Let's test that this works by first using a simple prompt.

In [ ]:
simple_prompt = 'How much is 2 + 2?'
response = openai.ChatCompletion.create(model='gpt-3.5-turbo',
                                        temperature=0, # degree of randomness of the output           
                                        messages=[{'role': 'user', 'content': simple_prompt}]
                                       )

Looks like it worked. Let's investigate the response. Notice that the output we are looking for is buried in field "choices".

In [ ]:
print(response.choices[0].message.content)

If we want just the result (i.e. 4), we should tell it that in the prompt. Try running above with prompt "How much is 2 + 2? Give me just the result".

By the way, we used model "gpt-3.5-turbo" which is just one of the ones available. You can see all of the options [here](https://platform.openai.com/docs/models), which is a list of models that at this stage still changes quite a lot. 

The list of messages we pass in allows us to also set up the context and behavior of the AI assistant. We can do so by specifying instructions using the "system" role as you can see in the example below.

In [ ]:
response = openai.ChatCompletion.create(model='gpt-3.5-turbo',
                                        temperature=0, # degree of randomness of the output           
                                        messages=[{'role': 'system', 'content': 'Be funny.'},
                                                  {'role': 'user', 'content': simple_prompt}]
                                       )
print(response.choices[0].message.content)

As was mentioned, the chat completion API can be used for multi-turn conversations. In this case a list of messages that contains the history of the chat needs to be passed as input, where the "user" role refers to the user prompts and the "assistant" refers to ChatGPT responses.

In [ ]:
messages = [{'role': 'user', 'content': 'Hi, who was the first US president?'},
            {'role': 'assistant', 'content': 'The first US president was George Washington.'},
            {'role': 'user', 'content': 'When was he born? And how about the second?'}]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=0
)

response.choices[0].message["content"]

## Accomplishing our gift agreements task

Alright, we are now ready to use the API on our task of extracting info from the gift contracts. Let's write two convenience functions first.

In [ ]:
def generate_response(prompt):
    """
    Query OpenAI API to get response.
    """

    response = openai.ChatCompletion.create(model='gpt-3.5-turbo',
                                            temperature=0, # degree of randomness of output           
                                            messages=[{'role': 'user', 'content': prompt}]
                                           )
                                
    
    return response.choices[0].message.content

In [ ]:
def generate_prompt(gift_contract_text):
    """
    Create prompt that gets sent to OpenAI API.
    """

    prompt = f'''Extract just the donor name, recipient name and the exact gift from 
                 the contract. Give the result as JSON with fields Donor, 
                 Recipient and Gift. If there are several gifts break it 
                 into a simple list of strings. {gift_contract_text}'''
    
    return prompt

We decided to ask ChatGPT to give us the results in the form of JSON so we could easily process the output further if we wanted to.

Let's try it on one contract.

In [ ]:
with open('data/Gift_Agreement_2431.txt', 'r') as f:
    contract_text = f.read()

In [ ]:
output = generate_response(generate_prompt(contract_text))

In [ ]:
import json
json.loads(output)

Looks good, we got what we wanted. We can now process all the gift agreements.

In [ ]:
from glob import glob

files = glob('data/*.txt')

In [ ]:
import time

def process_agreement(filename):
    time.sleep(15)
    with open(filename, 'r') as f:
        contract_text = f.read()
    
    output = generate_response(generate_prompt(contract_text))
    
    return json.loads(output)

In [ ]:
processed_agreements = [process_agreement(f) for f in files]

Notice that we did add `sleep` to the function. The reason for that is that at the time of this writing, the free version will allow you to make no more than 3 requests per minute if you are using the most cost effective model ("gpt-3.5-turbo"). 

In [ ]:
processed_agreements

## Example 2: Extracting names of people from text

Extracting names of people from some text is another example where ChatGPT shines.

In [ ]:
import ast

def gpt_parse(caption):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": f'''Who are the people in the following string?  
            Please answer in the form of a Python list of strings, without any commentary.  
            Each entry in the list should be each person's full name, as best you can determine it. 
            And remove people's titles.

    "{caption}"'''}
        ]
    )
    try:
        return ast.literal_eval(response['choices'][0]['message']['content'])
    except Exception as e:
        print(e)
        return []

In [ ]:
caption1 = 'Bob and Claire Rigel with Tom Hall'
print(caption1)
print(gpt_parse(caption1))

In [ ]:
caption2 = 'Board Chair Brooke Garber Neidich, Dr. Ruth Westheimer, President Harold S. Koplewicz, MD, Michael Fascitelli, Governor Jon S. Corzine, and Speaker Christine Quinn'
print(caption2)
print(gpt_parse(caption2))

## ChatGPT limitations and concerns

While we've seen examples where ChatGPT can be really useful and can make various NLP tasks easy, it also has some known practical limitations that you should be aware of before using it it your own work:

- Propensity for generating plausible-sounding but incorrect answers, also known as [AI hallucinations](https://decrypt.co/143064/openai-wants-to-stop-ai-from-hallucinating-and-lying).
- Limited knowledge and outdated information: ChatGPT's knowledge is based on pre-existing data up until September 2021. It might not have access to the most recent information or developments in various fields.
- [Bias](https://www.cbsnews.com/news/chatgpt-large-language-model-bias-60-minutes-2023-03-05/) and inappropriate responses: The training data used to develop ChatGPT contains biases present in the source material. Consequently, the model may generate biased or inappropriate responses, especially when prompted with sensitive or controversial topics.
- Lack of contextual understanding: It may sometimes provide inconsistent or incoherent responses, especially when presented with complex or multi-turn queries.
- Sensitivity to input phrasing: The model can be highly sensitive to the way a question is phrased. Slight rephrasing of the same question may yield different responses, which can be problematic when seeking consistent and accurate information.
- Security and privacy concerns: Sharing personal, sensitive, or confidential information with ChatGPT poses [potential risks](https://www.makeuseof.com/shouldnt-trust-chatgpt-confidential-data/). As an AI model, it does not have inherent privacy or security safeguards, and data shared with it could be stored or used in unintended ways.


We should also be aware that this new technology has some serious [limitations and potential dangers](https://www.makeuseof.com/openai-chatgpt-biggest-probelms/) to society. It poses legal and ethical issues related to copyright, privacy, [misuse](https://www.reuters.com/technology/europol-sounds-alarm-about-criminal-use-chatgpt-sees-grim-outlook-2023-03-27/), bias, and transparency. These models could for example be used for **large scale disinformation** in ways we can not begin to imagine. Or for example once the tool gets even better at writing code, it can be used for offensive **cyber attacks** on unprecendented scale. Another common concern is that the cost of training such AI makes it hard for anyone but the richest labs to build and train such models. This means we may be looking at a future where such powerful AI is in the hands of small corporate teams behind closed doors, without proper scrutiny or the input of a wider research community.

## Best practices

Here are a some of best practices to keep in mind when you are using ChatGPT in your own applications:

- If you are using the API, keep your API key safe and do not share it with anyone.
- Refrain from inputting any sensitive data, sensitive source code or any info that should not be disclosed. 
- Use specific prompts and iteratively improve them to make sure you are getting the results you are looking for.
- Have a human reviewer review, modify, and approve the system's outputs before they are used in production and delivered to end-users.
- If you are using ChatGPT to generate code for you, keep in mind that while it is able to write code, it won't be able to write code for very specific or complex problems that require specific experience and deep understanding. It is a good idea to break down your problem into smaller ones. And be ready to review and fix the code you'll be given before you can use it and have it work as intended. 
- Some advice from OpenAI regarding best safety practices can also be found [here](https://platform.openai.com/docs/guides/safety-best-practices).

*Copyright &copy; 2023 Pragmatic Institute. This content is licensed solely for personal use. Redistribution or publication of this material is strictly prohibited.*